<table style="width: 100%;">
    <tr>
        <td><a href="https://ieb-chile.cl/en/" target="_blank"><img src="https://raw.githubusercontent.com/IEB-BIODATA/pydwca-examples/main/images/logo/IEB.png" style="height: 100px;"></a</td>
        <td></td>
        <td><img src="https://raw.githubusercontent.com/IEB-BIODATA/pydwca-examples/main/images/logo/Biodata.png" style="height: 100px;"></td>
    </tr>
</table>

# Lazy Ingestion of GBIF occurrences

This notebook show how to ingest a Darwin Core Archive into a database using the Lazy Evaluation with [Polars](https://pola.rs/).

In [1]:
from tqdm.notebook import tqdm
from dwca import DarwinCoreArchive

In [2]:
gbif_chile = DarwinCoreArchive.from_file("data/chile-dwca-var.zip", lazy=True)
gbif_chile

/home/ubuntu/pydwca/src/dwca/classes/data_file.py:285: UserWarning: http://rs.tdwg.org/dwc/terms/geodeticDatum not in expected namespace for http://rs.tdwg.org/dwc/terms/Occurrence class. Some functionalities may not be available.
  warn(f"{element.get('term')} not in expected namespace for "
/home/ubuntu/pydwca/src/dwca/classes/data_file.py:312: SyntaxWarning: Error on field b'<field xmlns="http://rs.tdwg.org/dwc/text/" default="WGS84" term="http://rs.tdwg.org/dwc/terms/geodeticDatum"/>\n    ': int() argument must be a string, a bytes-like object or a real number, not 'NoneType'
  warn(f"Error on field {et.tostring(field_tree)}: {e}", category=SyntaxWarning)
/home/ubuntu/pydwca/src/dwca/classes/data_file.py:285: UserWarning: http://rs.gbif.org/terms/1.0/gbifID not in expected namespace for http://rs.tdwg.org/dwc/terms/Occurrence class. Some functionalities may not be available.
  warn(f"{element.get('term')} not in expected namespace for "
/home/ubuntu/pydwca/src/dwca/classes/data_fil

<Darwin Core Archive (10.15468/dl.p8492v [Core: http://rs.tdwg.org/dwc/terms/Occurrence, Entries: 10665951])>

## Connect to database

In [3]:
import os
import psycopg2
from dotenv import load_dotenv
from psycopg2.errors import ForeignKeyViolation

In [4]:
if load_dotenv():
    print("Environmental variables loaded")
else:
    print("Error importing env var")

Environmental variables loaded


In [5]:
connection = psycopg2.connect(
    host=os.getenv("LOCAL_HOST"),
    port=os.getenv("LOCAL_PORT"),
    user=os.getenv("LOCAL_USER"),
    password=os.getenv("LOCAL_PASS"),
    dbname=os.getenv("LOCAL_DB"),
)

In [6]:
connection.rollback()

### Create tables

In [7]:
with connection.cursor() as cursor:
    cursor.execute(gbif_chile.core.sql_table)
connection.commit()

Here we have two extension, but one of the same class of the core, so we are going to change that name by hand prior execute the query:

In [8]:
for extension in gbif_chile.extensions:
    print(extension)

Extension:
	class: http://rs.gbif.org/terms/1.0/Multimedia
	filename: multimedia.txt
	content: 610175 entries
Extension:
	class: http://rs.tdwg.org/dwc/terms/Occurrence
	filename: verbatim.txt
	content: 10665951 entries


In [9]:
with connection.cursor() as cursor:
    for extension in gbif_chile.extensions:
        cursor.execute(extension.sql_table.replace("CREATE TABLE \"Occurrence\"", "CREATE TABLE \"Verbatim\""))
connection.commit()

### Insert Data

Due to the long time this will take, here we show a sample.

In [10]:
from psycopg2.extras import execute_batch

In [11]:
batch_size = 1000

In [12]:
sample_size = 5

In [13]:
i = 0
with connection.cursor() as cursor:
    values = list()
    for statement, value in tqdm(gbif_chile.core.insert_sql, total=sample_size * batch_size):
        values.append(value)
        if len(values) >= batch_size:
            execute_batch(cursor, statement, values)
            values.clear()
            i += 1
        if i >= sample_size:
            break
    if len(values) > 0:
        execute_batch(cursor, statement, values)
        values.clear()
connection.commit()

  0%|          | 0/5000 [00:00<?, ?it/s]

Same issue with `"Verbatim"` extension. Change the statement to replace `"Occurrence"` for `"Verbatim"`

In [14]:
connection.autocommit = True

In [15]:
for extension in gbif_chile.extensions:
    i = 0
    with connection.cursor() as cursor:
        for statement, values in tqdm(extension.insert_sql, total=sample_size * batch_size):
            try:
                cursor.execute(statement.replace("INSERT INTO \"Occurrence\"", "INSERT INTO \"Verbatim\""), values)
            except ForeignKeyViolation:
                connection.rollback()
            i += 1
            if i > sample_size * batch_size:
                break
    connection.commit()

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [16]:
gbif_chile.core.close()
for extension in gbif_chile.extensions:
    extension.close()

The complete code as a script is available [here](gbif_chile_ingestion.py).

<table>
    <tr>
        <td colspan="3" style="text-align: center;"><p>BIODATA - <a href="https://ieb-chile.cl/en/" target="_blank">Institute of Ecology and Biodiversity</a> © 2024</p></td>
    </tr>
</table>